# Identification of the long roads in Bangladesh

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from haversine import haversine, Unit

In [ ]:
roads = pd.read_csv('../data/raw/_roads3.csv')
roads.rename(columns={'lrp':'LRPName'}, inplace=True)
roads.head(2)

In [ ]:
start_end = pd.DataFrame()

for r in roads['road'].unique():
    road_of_interest = roads[roads['road']==r]
    first_row = road_of_interest.head(1)
    last_row = road_of_interest.tail(1)
    start_end = pd.concat([start_end, first_row, last_row])
    
start_end = start_end.reset_index(drop=True)
start_end['model_type']= 'sourcesink'
start_end

In [ ]:
for i in start_end.index:
    start_end.at[i,'length']=start_end.at[i,'chainage']

In [ ]:
# The pool of roads, long enough to consider for this assignment
long_roads = start_end[start_end['chainage']>25]
long_roads_list = long_roads['road'].unique().tolist()
# long_roads

## Identify the roads crossing the N1 or the N2

In [ ]:
# We are interested in the crossings of the N1 and N2. For this reason we create a slice of the 'roads' dataframe that includes all the points of N1 and N2. In further steps we will inspect if these LRPs hint about existing intersections.
df_N1_N2 = roads[(roads['road']=='N1') | (roads['road']=='N2')]
# df_N1_N2.tail()

In [ ]:
# As a preparation step we will transform the type column into lower letters in order to filter more securely on certain words that might be referring to an intersection.
df_N1_N2['type'] = df_N1_N2['type'].str.lower()

# In order to keep track of all the LRPs that contain crossroads or side roads, we create two lists. By iterating over all the rows of the N1N2 slice of the 'roads' dataframe and checking if their 'type' contains either 'crossroad' or 'sideroad' we can keep track of the content of the column 'name' of those roads. The reason why we chose to collect the 'name' column is because it can contain information about the road with whom the N1 or N2 is intersecting. These roads we will try to abstract in further steps.

# Filter rows containing 'crossroad' or 'sideroad'
crossroad_df = df_N1_N2[df_N1_N2['type'].str.contains('crossroad')]
sideroad_df = df_N1_N2[df_N1_N2['type'].str.contains('sideroad')]

# Concatenate DataFrames
documented_intersections_N1N2 = pd.concat([crossroad_df, sideroad_df], ignore_index=True)

documented_intersections_N1N2

In [ ]:
# Now for the stored strings of the possible intersection LRPs we want to read out the roads. To do this we wrote the function split_list that splits each element of a list into separate string elements.
def split_list_df(string):
    split_step1 = string.split()
    split_list = []
    for i in split_step1:
        split_step2 = re.split(r'[(|i,)]', i)
        split_list.extend(split_step2)
    return split_list

# After that we flattened the nested lists into one big list of elements
def flatten_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

# Find the roads present in the lists, by checking if any of the characters of the elements in the list is a digit
def find_road_df(lst):
    return [element for element in lst if any(i.isdigit() for i in element)]

In [ ]:
# Use the functions on the dataframe of all intersection points to find out which road they are leading to and store that in the column: road_to
documented_intersections_N1N2['road_to'] = documented_intersections_N1N2['name'].apply(lambda x: find_road_df(split_list_df(x)))

for i in documented_intersections_N1N2.index:
    if documented_intersections_N1N2.at[i, 'road_to']:  # Check!! Otherwise the stupid out of index error
        documented_intersections_N1N2.at[i, 'road_to'] = documented_intersections_N1N2.at[i, 'road_to'][0]
    else:
        documented_intersections_N1N2.at[i, 'road_to'] = None
        
documented_intersections_N1N2

In [ ]:
# Now that we know to which road they are intersecting to, we have to make sure we only consider the long roads.
documented_long_intersections_N1N2 = documented_intersections_N1N2[documented_intersections_N1N2['road_to'].isin(long_roads_list)]
documented_long_intersections_N1N2

In [ ]:
plt.figure(figsize=(10,6)) 
for road in documented_long_intersections_N1N2.road_to:
    plt.plot(roads[roads['road']==road]['lon'], roads[roads['road']==road]['lat'], label=road)

# # Plot N1 (not included in its own intersections)
plt.plot(roads[roads['road']=='N1']['lon'], roads[roads['road']=='N1']['lat'], label='N1')

# Make the intersections big enough and red
plt.scatter(documented_long_intersections_N1N2['lon'], documented_long_intersections_N1N2['lat'], color='red', s=100, label='Intersections')  

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc="lower left", ncol=len(documented_long_intersections_N1N2.columns), bbox_to_anchor=(-0.1, -0.25),
          fancybox=True, shadow=True) 
plt.title('Documented intersections of N1N2')

## Extract remaining roads for additional check
The 30 identified roads were all identified based on their type (either CrossRoad or SideRoad). It is however also possible that the provided dataset is incomplete to this regard. Therefor, the remaining long roads (>25km), will be extracted to a new dataframe. The 243 roads in this file will be visually inspected on possible crossings with the N1 and N2. For this analysis, see notebook 'check_remaining_crossings'. 

In [ ]:
# Extract all of the roads longer than 25km, remove the already identified roads from the list and make a dataframe with the remaining ones
documented_crossings_list = documented_long_intersections_N1N2['road_to'].to_list()
possible_crossings_list = ['N1', 'N2'] + list(set(long_roads_list) - set(documented_crossings_list))
df_possible_crossings = roads[roads['road'].isin(possible_crossings_list)]
df_possible_crossings.to_csv('../data/semi-processed/possible_crossings_to_check.csv')

## Calculate the closest distance from N1 or N2 to the intersecting road

In [ ]:
documented_intersections = documented_long_intersections_N1N2.copy()

# Create empty lists to store closest point information
distances_to_closest = []

# Iterate over each row in find_points DataFrame
for index, row in documented_long_intersections_N1N2.iterrows():
    # Extract coordinates of the point
    point_coords = (row['lat'], row['lon'])
    
    # Filter roads df to get points on road_to
    road_to_points = roads[roads['road'] == row['road_to']]
    
    # Calculate distances between the point and all road_to points using Haversine formula
    distances = [haversine(point_coords, (lat, lon), unit=Unit.METERS) for lat, lon in zip(road_to_points['lat'], road_to_points['lon'])]
    
    # Find the index of the closest point
    closest_point_index = distances.index(min(distances))
    
    # Calculate distance between the two points
    distance_between = distances[closest_point_index]
    
    # Append closest point information to lists
    distances_to_closest.append(distance_between)

# Add new columns to find_points DataFrame
documented_intersections['distance_to_closest'] = distances_to_closest

documented_intersections

In [ ]:
documented_intersections.distance_to_closest.sort_values(ascending=False)

In [ ]:
## Threshold is arbitrairy on 100 meters, because we checked the one on 66 meter and it looks like they overlap in reality
intersection_threshold = 100

In [ ]:
trustworthy_documented_intersections = documented_intersections[documented_intersections['distance_to_closest']< intersection_threshold]
trustworthy_documented_intersections

In [ ]:
def find_closest(row, roads):
    intersection_coords = (row['lat'], row['lon'])
    road_to_points = roads[roads['road'] == row['road_to']]

    # Calculate distances between the intersection and all road_to points using haversine
    distances = [haversine(intersection_coords, (lat, lon), unit=Unit.METERS) for lat, lon in zip(road_to_points['lat'], road_to_points['lon'])] 
    
    # Find the index of the closest point
    closest_point_index = distances.index(min(distances))
    
    # Get the coordinates and LRPName of the closest point
    closest_lat = road_to_points.iloc[closest_point_index]['lat']
    closest_lon = road_to_points.iloc[closest_point_index]['lon']
    closest_lrp_name = road_to_points.iloc[closest_point_index]['LRPName']
    closest_name = road_to_points.iloc[closest_point_index]['name']
    closest_type= road_to_points.iloc[closest_point_index]['type']
    closest_chainage = road_to_points.iloc[closest_point_index]['chainage']
    
    # Calculate distance to the closest point
    distance_to_closest = distances[closest_point_index]
    
    return closest_lat, closest_lon, closest_lrp_name, closest_name, closest_type, closest_chainage, distance_to_closest

# Apply the function to each row of trustworthy_documented_intersections_df
closest_cols = trustworthy_documented_intersections.apply(lambda row: find_closest(row, roads), axis=1)

# Unpack the result into separate columns
trustworthy_documented_intersections[['closest_lat', 'closest_lon', 'closest_lrp', 'closest_name', 'closest_type', 'closest_chainage', 'distance_to_closest']] = pd.DataFrame(closest_cols.tolist(), index=trustworthy_documented_intersections.index)

# Set 'model_type' to 'intersection' for all rows
trustworthy_documented_intersections['model_type'] = 'intersection'

# Print or use the trustworthy_documented_intersections DataFrame as needed
trustworthy_documented_intersections

In [ ]:
trustworthy_documented_intersections.to_csv('../data/semi-processed/intersections_to_check_final.csv')